# Load libraries

In [1]:
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras.models import load_model
import tensorflow.compat.v1.keras.backend as K
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import ListedColormap

In [3]:
import cv2
import glob
import time
import gc

# Functions

In [4]:
def get_iterate(model, layer):
    # Get the output of the layer and the data
    data_output = model.output[:, 0]

    # Get layer output
    get_layer = model.get_layer(layer)
    layer_output = get_layer.output

    # Calculate the gradients
    grads = K.gradients(data_output, layer_output)[0]

    pooled_grads = K.mean(grads, axis = (0, 1))

    iterate = K.function([model.input], [pooled_grads, layer_output[0]])
    
    return(iterate)

In [5]:
def get_heatmap_matrix(dataset, nth_data, iterate):

    # Get the data
    data = dataset[nth_data:nth_data+1, :, :]
    
    pooled_grads_value, layer_output_value = iterate([data])

    layer_output_value[:, ] *= pooled_grads_value
        
    # Get the heatmap matrix
    heatmap = np.copy(layer_output_value)
    heatmap = np.mean(heatmap, axis = -1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    # Expand the dimensionality of heatmap so that it can be plot
    heatmap = np.expand_dims(heatmap, axis=0)
    
    return heatmap

In [6]:
# Original function
# def load_npy_data(filename):
#     # Define pattern of the npy data
#     data_loading_pattern = "data-sets/" + filename + ".npy"
#     data_loading_path = glob.glob(data_loading_pattern)

#     # Load the data
#     print("Loading data:", str(filename), "from " + data_loading_path[0])
#     data = np.load(data_loading_path[0])
    
#     return(data)

# Define function for secreted data
def load_npy_data(filename):
    # Define pattern of the npy data
    data_loading_pattern = "../../../data/secreted_data/ready_to_process/encoded_files/" + filename + ".npy"
    data_loading_path = glob.glob(data_loading_pattern)

    # Load the data
    print("Loading data:", str(filename), "from " + data_loading_path[0])
    data = np.load(data_loading_path[0])
    
    return(data)

In [7]:
def get_sum_heatmap(class_name, data_name, model, layer, from_sample, sample_length):
    
    file_name = class_name + "/" + data_name
    data = load_npy_data(file_name)
    print("Data shape: " + str(data.shape[0]))
    
    iterate = get_iterate(model, layer)
    result = []

    min_sample = from_sample
    max_sample = np.min([from_sample + sample_length, data.shape[0]])
    
    print("")
    # For loop to get the heatmap for each matrix
    for nth_sample in range(min_sample, max_sample):
        print("Getting the heatmap for the data: sample " + str(nth_sample) + "/" + str(max_sample-1))
        heatmap = get_heatmap_matrix(data, nth_sample, iterate)

        # Put all of the results together
        result.append(heatmap)
        
        # Free memory (useless)
        gc.collect()

    # Change the list to numpy array
    all_matrices = np.array(result)

    # Save all matrices
    saving_path = "results_secreted/all_matrices_" + class_name + "_" + data_name + "_" + layer + "_samples_" + str(min_sample).zfill(4) + "_" + str(max_sample-1).zfill(4) + ".npy"

    print("")
    print("Saving results in", saving_path)
    np.save(saving_path, all_matrices)

# Run functions in steps

In [10]:
# model = load_model("models/oomycete/cnn-lstm/model_1.30-0.30.hdf5")
model = load_model("../../../data/secreted_data/saved_models/oomycete/cnn_gru/sequential_1.30-0.43.hdf5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 933, 16)           656       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 466, 16)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 16)                1200      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [24]:
get_sum_heatmap(
    class_name = "oomycete",
    data_name = "x_train_oomycete", 
    model = model,  
    layer = "conv1d_1",
    from_sample = 100,
    sample_length = 25
)

Loading data: oomycete/x_train_oomycete from ../../../data/secreted_data/ready_to_process/encoded_files/oomycete/x_train_oomycete.npy
Data shape: 102

Getting the heatmap for the data: sample 100/101
Getting the heatmap for the data: sample 101/101

Saving results in results_secreted/all_matrices_oomycete_x_train_oomycete_conv1d_1_samples_0100_0101.npy


In [18]:
get_sum_heatmap(
    class_name = "oomycete",
    data_name = "x_val_oomycete", 
    model = model,  
    layer = "conv1d_1",
    from_sample = 0,
    sample_length = 25
)

Loading data: oomycete/x_val_oomycete from ../../../data/secreted_data/ready_to_process/encoded_files/oomycete/x_val_oomycete.npy
Data shape: 34

Getting the heatmap for the data: sample 0/33
Getting the heatmap for the data: sample 1/33
Getting the heatmap for the data: sample 2/33
Getting the heatmap for the data: sample 3/33
Getting the heatmap for the data: sample 4/33
Getting the heatmap for the data: sample 5/33
Getting the heatmap for the data: sample 6/33
Getting the heatmap for the data: sample 7/33
Getting the heatmap for the data: sample 8/33
Getting the heatmap for the data: sample 9/33
Getting the heatmap for the data: sample 10/33
Getting the heatmap for the data: sample 11/33
Getting the heatmap for the data: sample 12/33
Getting the heatmap for the data: sample 13/33
Getting the heatmap for the data: sample 14/33
Getting the heatmap for the data: sample 15/33
Getting the heatmap for the data: sample 16/33
Getting the heatmap for the data: sample 17/33
Getting the heatma

In [19]:
get_sum_heatmap(
    class_name = "oomycete",
    data_name = "x_test_oomycete", 
    model = model,  
    layer = "conv1d_1",
    from_sample = 0,
    sample_length = 20
)

Loading data: oomycete/x_test_oomycete from ../../../data/secreted_data/ready_to_process/encoded_files/oomycete/x_test_oomycete.npy
Data shape: 34

Getting the heatmap for the data: sample 0/33
Getting the heatmap for the data: sample 1/33
Getting the heatmap for the data: sample 2/33
Getting the heatmap for the data: sample 3/33
Getting the heatmap for the data: sample 4/33
Getting the heatmap for the data: sample 5/33
Getting the heatmap for the data: sample 6/33
Getting the heatmap for the data: sample 7/33
Getting the heatmap for the data: sample 8/33
Getting the heatmap for the data: sample 9/33
Getting the heatmap for the data: sample 10/33
Getting the heatmap for the data: sample 11/33
Getting the heatmap for the data: sample 12/33
Getting the heatmap for the data: sample 13/33
Getting the heatmap for the data: sample 14/33
Getting the heatmap for the data: sample 15/33
Getting the heatmap for the data: sample 16/33
Getting the heatmap for the data: sample 17/33
Getting the heat